In [2]:
import pandas as pd
import numpy as np


file_path = 'bank-full-clean.csv'
df = pd.read_csv(file_path)


print(df.info())
print(df.describe(include='all'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB
None
                 age          job  marital  education default        balance  \
count   45211.

In [4]:
# 2. Handling missing values

df.replace([-2, -88], np.nan, inplace=True)

# Fill in categorical variables (with mode or 'unknown')
categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
for col in categorical_cols:
    mode = df[col].mode()[0]
    df[col].fillna(mode, inplace=True)

# Filling numeric variables (using median)
numeric_cols = ['balance', 'day', 'duration', 'campaign']
for col in numeric_cols:
    median = df[col].median()
    df[col].fillna(median, inplace=True)
    

C:\Users\华硕\AppData\Local\Temp\ipykernel_20228\4074447170.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode, inplace=True)
C:\Users\华硕\AppData\Local\Temp\ipykernel_20228\4074447170.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing '

In [6]:
# 3. Outlier Detection (Demonstration: Using IQR)
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"{col} Number of outliers：{len(outliers)}")
    df.loc[(df[col] < lower_bound) | (df[col] > upper_bound), col] = np.nan

for col in numeric_cols:
    median = df[col].median()
    df[col].fillna(median, inplace=True)

balance 异常值数：4733
day 异常值数：0
duration 异常值数：3235
campaign 异常值数：3064


C:\Users\华硕\AppData\Local\Temp\ipykernel_20228\1203577414.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median, inplace=True)


In [8]:
#drop_duplicates
df.drop_duplicates(inplace=True)

In [10]:
print(df.columns)

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')


In [12]:
# One-hot encode 'marital', 'job', 'education', 'contact', and 'poutcome' to ensure that no category is omitted
categorical_cols = ['marital', 'job', 'education', 'contact', 'poutcome']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=False)

In [14]:
# 7. Normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for col in ['balance', 'day', 'duration', 'campaign']:
    df[col] = scaler.fit_transform(df[[col]])


In [15]:
import pandas as pd
# 1. Define a dictionary mapping months to seasons
month_to_season_map = {
    'jan': 'winter',
    'feb': 'winter',
    'mar': 'spring',
    'apr': 'spring',
    'may': 'spring',
    'jun': 'summer',
    'jul': 'summer',
    'aug': 'summer',
    'sep': 'autumn',
    'oct': 'autumn',
    'nov': 'autumn',
    'dec': 'winter'
}

# 2.Apply mapping to generate season column
df['season'] = df['month'].str.lower().map(month_to_season_map)

# 3. Generate one-hot encodings of all four seasons (making sure not to drop any categories)
season_dummies = pd.get_dummies(df['season'], prefix='season', drop_first=False)

# 4. Add seasonal dummy variables to the original data
df = pd.concat([df, season_dummies], axis=1)

# This will give you four columns: season_winter, season_spring, season_summer, season_autumn, each with a value of 0 or 1


In [18]:
# 1. Age 4 categories (youth, middle-aged, middle-aged and elderly, elderly)
# Hypothetical categories: youth (19-35), middle-aged (36-50), middle-aged and elderly (51-65), elderly (66-87)
df['age_19_35']   = ((df['age'] >= 19) & (df['age'] <= 35)).astype(int)
df['age_36_50']   = ((df['age'] > 35) & (df['age'] <= 50)).astype(int)
df['age_51_65']   = ((df['age'] > 50) & (df['age'] <= 65)).astype(int)
df['age_66_87']   = (df['age'] > 65).astype(int)

# 2. Balance 4 levels (low, medium, high, high)
df['balance_low']     = (df['balance'] < -1).astype(int)
df['balance_medium']  = ((df['balance'] >= -1) & (df['balance'] < 0)).astype(int)
df['balance_high']    = ((df['balance'] >= 0) & (df['balance'] < 1)).astype(int)
df['balance_very_high']= (df['balance'] >= 1).astype(int)

# 3. Duration 4levels (low, medium, high, high)
df['duration_short']   = (df['duration'] <= -0.5).astype(int)
df['duration_medium']= ((df['duration'] > -0.5) & (df['duration'] < 1)).astype(int)
df['duration_long']  = ((df['duration'] >= 1) & (df['duration'] < 2)).astype(int)
df['duration_very_long'] = (df['duration'] >= 2).astype(int)

df['campaign_-inf_-0.3'] = (df['campaign'] <= -0.3).astype(int)
df['campaign_-0.3_0'] = ((df['campaign'] > -0.3) & (df['campaign'] <= 0)).astype(int)
df['campaign_0_1'] = ((df['campaign'] > 0) & (df['campaign'] <= 1)).astype(int)
df['campaign_1_inf'] = (df['campaign'] > 1).astype(int)


# 5. Split pdays into:
# a) Whether to contact (pdays == -1 means no contact, otherwise 1)
# b) Segments: 0-300, 301-871
df['pdays_not_contact'] = (df['pdays'] == -1).astype(int)
df['pdays_contact_range1'] = ((df['pdays'] > 0) & (df['pdays'] <=300)).astype(int)
df['pdays_contact_range2'] = ((df['pdays'] > 300) & (df['pdays'] <=871)).astype(int)

# 6. Previous times (number of previous contacts)
df['previous_0'] = (df['previous'] == 0).astype(int)
df['previous_1_5'] = ((df['previous'] >= 1) & (df['previous'] <=5)).astype(int)
df['previous_6_15'] = ((df['previous'] > 5) & (df['previous'] <=15)).astype(int)
df['previous_16_plus'] = (df['previous'] >15).astype(int)

# 7. Convert categorical features (already yes/no) to 0/1
for col in ['default', 'housing', 'loan']:
    df[col] = df[col].map({'no':0, 'yes':1})

In [20]:
df = df.replace({True: 1, False: 0})

C:\Users\华硕\AppData\Local\Temp\ipykernel_20228\404666328.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


In [22]:
df['y'] = df['y'].map({'no': 0, 'yes': 1})

In [24]:
print(df.info())

df.to_csv('prepared_bank_data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 66 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   age                   45211 non-null  int64  
 1   default               45211 non-null  int64  
 2   balance               45211 non-null  float64
 3   housing               45211 non-null  int64  
 4   loan                  45211 non-null  int64  
 5   day                   45211 non-null  float64
 6   month                 45211 non-null  object 
 7   duration              45211 non-null  float64
 8   campaign              45211 non-null  float64
 9   pdays                 45211 non-null  int64  
 10  previous              45211 non-null  int64  
 11  y                     45211 non-null  int64  
 12  marital_divorced      45211 non-null  int64  
 13  marital_married       45211 non-null  int64  
 14  marital_single        45211 non-null  int64  
 15  job_admin.         